In [8]:
import fastf1
import pandas as pd

# Enable caching for faster data loading
fastf1.Cache.enable_cache('cache') 

In [9]:
# Import the results_df dataset
results_df = pd.read_csv('model/DATA/f1_results_2024_2025.csv')

In [10]:
# Load driver data from an Excel file
drivers = pd.read_excel('model/DATA/drivers_experience.xlsx')
drivers

,Abbreviation,FullName,TeamId,DriverExperience
0,NOR,Lando Norris,mclaren,51
1,VER,Max Verstappen,red_bull,51
2,RUS,George Russell,mercedes,51
3,ANT,Andrea Kimi Antonelli,mercedes,1
4,ALB,Alexander Albon,williams,31
5,STR,Lance Stroll,aston_martin,49
6,HUL,Nico Hulkenberg,sauber,17
7,LEC,Charles Leclerc,ferrari,51
8,PIA,Oscar Piastri,mclaren,15
9,HAM,Lewis Hamilton,ferrari,50


In [11]:
filtered_results = results_df[results_df["Abbreviation"].isin(drivers['Abbreviation'])]

# Group by driver and sum total points
total_points = filtered_results.groupby("Abbreviation")["Points"].sum().reset_index()

# Fill the missing rookie drivers with 0 points
all_drivers_df = pd.DataFrame({"Abbreviation": drivers["Abbreviation"]}) 
total_points = all_drivers_df.merge(total_points, on="Abbreviation", how="left").fillna(0) 

# Convert Points to absolute value interger for simpler point system
total_points["Points"] = total_points["Points"].astype(int)
total_points = total_points.sort_values(by="Points", ascending=False)

In [12]:
total_points

,Abbreviation,Points
1,VER,738
0,NOR,711
8,PIA,610
7,LEC,536
2,RUS,497
9,HAM,338
17,SAI,301
3,ANT,125
16,ALO,107
6,HUL,85


In [13]:
drivers = total_points.merge(drivers, on="Abbreviation", how="left").fillna(0) 
drivers

,Abbreviation,Points,FullName,TeamId,DriverExperience
0,VER,738,Max Verstappen,red_bull,51
1,NOR,711,Lando Norris,mclaren,51
2,PIA,610,Oscar Piastri,mclaren,15
3,LEC,536,Charles Leclerc,ferrari,51
4,RUS,497,George Russell,mercedes,51
5,HAM,338,Lewis Hamilton,ferrari,50
6,SAI,301,Carlos Sainz,williams,51
7,ANT,125,Andrea Kimi Antonelli,mercedes,1
8,ALO,107,Fernando Alonso,aston_martin,34
9,HUL,85,Nico Hulkenberg,sauber,17


In [14]:
# Calculate average qualifying position for each driver
avg_quali_df = results_df.groupby('Abbreviation')['GridPosition'].mean().reset_index()
avg_quali_df.rename(columns={'GridPosition': 'AvgQualiPosition'}, inplace=True)

In [15]:
# Calculate average race position for each driver
avg_race_df = results_df.groupby('Abbreviation')['Position'].mean().reset_index()
avg_race_df.rename(columns={'Position': 'AvgRacePosition'}, inplace=True)

In [16]:
# Merge both dataframes
avg_pos_df = avg_quali_df.merge(avg_race_df, on='Abbreviation', how='outer')

# Merge the drivers DataFrame with avg_pos_df to include AvgQualiPosition and AvgRacePosition
drivers = drivers.merge(avg_pos_df, on='Abbreviation', how='left')

# Display updated drivers dataset
drivers

,Abbreviation,Points,FullName,TeamId,DriverExperience,AvgQualiPosition,AvgRacePosition
0,VER,738,Max Verstappen,red_bull,51,3.630435,3.760870
1,NOR,711,Lando Norris,mclaren,51,3.217391,4.304348
2,PIA,610,Oscar Piastri,mclaren,15,4.304348,4.782609
3,LEC,536,Charles Leclerc,ferrari,51,5.326087,5.630435
4,RUS,497,George Russell,mercedes,51,5.043478,5.652174
5,HAM,338,Lewis Hamilton,ferrari,50,8.782609,7.630435
6,SAI,301,Carlos Sainz,williams,51,8.444444,9.044444
7,ANT,125,Andrea Kimi Antonelli,mercedes,1,8.681818,9.818182
8,ALO,107,Fernando Alonso,aston_martin,34,9.934783,11.217391
9,HUL,85,Nico Hulkenberg,sauber,17,13.239130,12.108696


In [17]:
# Remove the 'Drivers'sExperince ' column from the drivers DataFrame as it's not needed
filtered_drivers_info = drivers.drop(['DriverExperience'], axis=1)
filtered_drivers_info

,Abbreviation,Points,FullName,TeamId,AvgQualiPosition,AvgRacePosition
0,VER,738,Max Verstappen,red_bull,3.630435,3.760870
1,NOR,711,Lando Norris,mclaren,3.217391,4.304348
2,PIA,610,Oscar Piastri,mclaren,4.304348,4.782609
3,LEC,536,Charles Leclerc,ferrari,5.326087,5.630435
4,RUS,497,George Russell,mercedes,5.043478,5.652174
5,HAM,338,Lewis Hamilton,ferrari,8.782609,7.630435
6,SAI,301,Carlos Sainz,williams,8.444444,9.044444
7,ANT,125,Andrea Kimi Antonelli,mercedes,8.681818,9.818182
8,ALO,107,Fernando Alonso,aston_martin,9.934783,11.217391
9,HUL,85,Nico Hulkenberg,sauber,13.239130,12.108696


In [18]:
filtered_drivers_info.to_csv('model/DATA/filtered_drivers_info.csv')

In [19]:
# Select only relevant features from results_df for further processing
filtered_results_df = results_df[['Round', 'Abbreviation', 'GridPosition', 'Position']]
filtered_results_df

,Round,Abbreviation,GridPosition,Position
0,1,VER,1.0,1.0
1,1,SAI,4.0,3.0
2,1,LEC,2.0,4.0
3,1,RUS,3.0,5.0
4,1,NOR,7.0,6.0
...,...,...,...,...
787,22,ALB,16.0,16.0
788,22,BOR,18.0,17.0
789,22,STR,12.0,18.0
790,22,NOR,1.0,19.0


In [20]:
# Merge the filtered race results with driver information using 'Abbreviation' as the key
# This combines race performance with driver statistics and gives the final dataset to use for prediction
final_df = filtered_results_df.merge(filtered_drivers_info, on='Abbreviation', how='left')
final_df

,Round,Abbreviation,GridPosition,Position,Points,FullName,TeamId,AvgQualiPosition,AvgRacePosition
0,1,VER,1.0,1.0,738.0,Max Verstappen,red_bull,3.630435,3.760870
1,1,SAI,4.0,3.0,301.0,Carlos Sainz,williams,8.444444,9.044444
2,1,LEC,2.0,4.0,536.0,Charles Leclerc,ferrari,5.326087,5.630435
3,1,RUS,3.0,5.0,497.0,George Russell,mercedes,5.043478,5.652174
4,1,NOR,7.0,6.0,711.0,Lando Norris,mclaren,3.217391,4.304348
...,...,...,...,...,...,...,...,...,...
787,22,ALB,16.0,16.0,82.0,Alexander Albon,williams,12.695652,12.760870
788,22,BOR,18.0,17.0,19.0,Gabriel Bortoleto,sauber,14.318182,14.454545
789,22,STR,12.0,18.0,52.0,Lance Stroll,aston_martin,14.022222,13.533333
790,22,NOR,1.0,19.0,711.0,Lando Norris,mclaren,3.217391,4.304348


In [21]:
final_df["QualifyingScore"] = (final_df["AvgQualiPosition"] + final_df["GridPosition"]) / 2
final_df

,Round,Abbreviation,GridPosition,Position,Points,FullName,TeamId,AvgQualiPosition,AvgRacePosition,QualifyingScore
0,1,VER,1.0,1.0,738.0,Max Verstappen,red_bull,3.630435,3.760870,2.315217
1,1,SAI,4.0,3.0,301.0,Carlos Sainz,williams,8.444444,9.044444,6.222222
2,1,LEC,2.0,4.0,536.0,Charles Leclerc,ferrari,5.326087,5.630435,3.663043
3,1,RUS,3.0,5.0,497.0,George Russell,mercedes,5.043478,5.652174,4.021739
4,1,NOR,7.0,6.0,711.0,Lando Norris,mclaren,3.217391,4.304348,5.108696
...,...,...,...,...,...,...,...,...,...,...
787,22,ALB,16.0,16.0,82.0,Alexander Albon,williams,12.695652,12.760870,14.347826
788,22,BOR,18.0,17.0,19.0,Gabriel Bortoleto,sauber,14.318182,14.454545,16.159091
789,22,STR,12.0,18.0,52.0,Lance Stroll,aston_martin,14.022222,13.533333,13.011111
790,22,NOR,1.0,19.0,711.0,Lando Norris,mclaren,3.217391,4.304348,2.108696


In [22]:
final_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 792 entries, 0 to 791
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Round             792 non-null    int64  
 1   Abbreviation      792 non-null    object 
 2   GridPosition      792 non-null    float64
 3   Position          792 non-null    float64
 4   Points            785 non-null    float64
 5   FullName          785 non-null    object 
 6   TeamId            785 non-null    object 
 7   AvgQualiPosition  785 non-null    float64
 8   AvgRacePosition   785 non-null    float64
 9   QualifyingScore   785 non-null    float64
dtypes: float64(6), int64(1), object(3)
memory usage: 62.0+ KB


In [23]:
# Count and find if there are any null values in the dataset
final_df.isnull().sum()

Round               0
Abbreviation        0
GridPosition        0
Position            0
Points              7
FullName            7
TeamId              7
AvgQualiPosition    7
AvgRacePosition     7
QualifyingScore     7
dtype: int64

In [24]:
df_clean = final_df.dropna()

In [25]:

df_clean.isnull().sum()

Round               0
Abbreviation        0
GridPosition        0
Position            0
Points              0
FullName            0
TeamId              0
AvgQualiPosition    0
AvgRacePosition     0
QualifyingScore     0
dtype: int64

In [26]:
df_clean.to_csv('model/DATA/f1_final_data.csv', index=False)